In [ ]:
import imp
import plotto
#imp.reload(plotto)
import pandas as pd
pd.set_option('display.max_colwidth', -1)

%matplotlib inline
#notebook
import matplotlib.units 

%run plots_test.ipynb

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#imp.reload(matplotlib)
import plotto

#from plots_test import *
#import plots_test
import plotto
ureg = plotto.ureg()

simul = plotto.query("/tmp/antonio/magneticum.sql.faster")
hubble=  simul("select h from simulation").h[0]
display(hubble)
#ureg.define('hubble = %f'%hubble)
clash  = plotto.ObservativeTable('data/CLASH.txt',csv_delimiter=',')


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }

In [201]:
simul ("""
            select snap.id as snap_id,*
            from simulation
            inner join snap on snap.simulation_id=simulation.id
            inner join (
                select fof.snap_id as id,count(*) as count
                from fof
                inner join pp
                on pp.id_cluster = fof.id_cluster and fof.snap_id = pp.snap_id
                where   fof.glen>1e4
                group by fof.snap_id
            ) as A on A.id = snap.id
       """)


,snap_id,id,name,box_size,h,id,name,redshift,a,simulation_id,tag,id,count
0,5,2,/HydroSims/Magneticum/Box0/mr_bao/,2688000.0,0.704,5,037,2.220446e-16,1.000000,2,z0,5,44537
1,6,2,/HydroSims/Magneticum/Box0/mr_bao/,2688000.0,0.704,6,025,4.701941e-01,0.680182,2,z05,6,11311
2,7,2,/HydroSims/Magneticum/Box0/mr_bao/,2688000.0,0.704,7,014,1.178895e+00,0.458948,2,z1,7,486
3,8,2,/HydroSims/Magneticum/Box0/mr_bao/,2688000.0,0.704,8,012,1.477581e+00,0.403619,2,z15,8,80
4,9,2,/HydroSims/Magneticum/Box0/mr_bao/,2688000.0,0.704,9,010,1.980082e+00,0.335561,2,z2,9,9
5,10,3,/HydroSims/Magneticum/Box2b/hr_bao/,640000.0,0.704,10,031,2.520891e-01,0.798665,3,z0,10,147552
6,11,3,/HydroSims/Magneticum/Box2b/hr_bao/,640000.0,0.704,11,026,4.701941e-01,0.680182,3,z05,11,137030
7,12,3,/HydroSims/Magneticum/Box2b/hr_bao/,640000.0,0.704,12,015,1.178895e+00,0.458948,3,z1,12,99669
8,13,3,/HydroSims/Magneticum/Box2b/hr_bao/,640000.0,0.704,13,012,1.706354e+00,0.369501,3,z15,13,63542
9,14,3,/HydroSims/Magneticum/Box2b/hr_bao/,640000.0,0.704,14,011,1.980082e+00,0.335561,3,z2,14,48925


In [ ]:
all_df_z0 = simul ("""
    select  fof.mcri, fof.tgas0, pp.* , fof.r5cc, fof.rvir, fof.m5cc, fof.glen, fof.rcri, fof.id_cluster as fof_id_cluster, pp.id_cluster as pp_id_cluster
    from pp
    inner join fof on fof.snap_id=pp.snap_id and fof.id_cluster = pp.id_cluster
    inner join snap on  pp.snap_id = snap.id
    inner join simulation on simulation.id = snap.simulation_id
    and instr( simulation.name,'_dm')<=0
    and fof.glen>1e4
   -- and pp.fossilness>=0
   -- and pp.fossilness< 
    and snap.tag='z0'
    --and snap.id= 5
    """)



In [ ]:
n_subsample = 10000
rdf = all_df_z0.sample(n=n_subsample)
rdf['c200c_rho0rs']=rdf.c200c_rho0/rdf.c200c_rs
f,a12 = plt.subplots(1,2, sharey=True, figsize=(15,5))
a1=a12[0]
a2=a12[1]
f.subplots_adjust(hspace=0)
a1.scatter( (ureg.gmass*rdf.mcri).to('Msun'),rdf.c200c, c=np.log10(rdf.c200c_rho0rs), s=2) #c=rdf.redshift, s=2)
a1.set_xscale('log')
a1.set_yscale('log')
a1.set_xlabel("Mcri [Msun] (color=log rho0)")
a1.set_ylabel("c")

a2.scatter( (ureg.gmass*rdf.mcri).to('Msun'),rdf.c200c, c=np.log10(rdf.c200c_rs), s=2) #c=rdf.redshift, s=2)
a2.set_xscale('log')
a2.set_yscale('log')
a2.set_xlabel("Mcri [Msun] (color=log rs)")
#a2.set_ylabel("c")


#f.text(.5, -0.05, "Boxes 0_mr, 2b_hr, 4_uhr @ z=0, subsample of %.0e objects"%(n_subsample), ha='right')
f.tight_layout()
f.subplots_adjust(wspace=0, hspace=0)

f,a12 = plt.subplots(1,2, figsize=(15,4))
a1=a12[0]
a2=a12[1]
mmask = (rdf.mcri>1e3) & (rdf.mcri<2e3)
a1.scatter( rdf[mmask].c200c_rs,rdf[mmask].c200c, c=np.log10(rdf[mmask].c200c_rho0rs), s=2) #c=rdf.redshift, s=2)
a1.set_xscale('log')
a1.set_yscale('log')
a1.set_xlabel("rs (color=log rho0)")
a1.set_ylabel("c, objects 1e13<M<2e13")
a2.scatter( rdf[mmask].c200c_rho0rs,rdf[mmask].c200c, c= np.log10(rdf[mmask].c200c_rs), s=2) #c=rdf.redshift, s=2)
a2.set_xscale('log')
a2.set_yscale('log')
a2.set_xlabel("rho0 (color=log rs)")
f.tight_layout()
f.subplots_adjust(wspace=0, hspace=0)

f,a = plt.subplots(1,1, figsize=(15,4))
mmask = (rdf.mcri>1e2) #(rdf.mcri>1e3) & (rdf.mcri<2e3)
#a.scatter( rdf[mmask].c200c_rho0rs,rdf[mmask].c200c_rs,  s=2) #c=rdf.redshift, s=2)
#a.scatter( rdf[mmask].c200c_rho0rs,rdf[mmask].c200c_rs,c=np.log10(rdf[mmask].c200c), s=2) #c=rdf.redshift, s=2)
a.set_xscale('log')
a.set_yscale('log')
a.scatter( rdf.c200c_rho0rs,rdf.c200c_rs, c=np.log10(rdf.mcri),  s=2) #c=rdf.redshift, s=2)
a.set_xlabel("rho0 (color=log c)")
a.set_xlabel("rs (color=log c)")
#a.set_xlim([1.e-4,1.])
f.tight_layout()
f.subplots_adjust(wspace=0, hspace=0)


None

In [ ]:
for idx, row in clash.iterrows():
    display(row.m200c.value/ row.m200c.value.to('Msun'))


In [ ]:
fig,a = plt.subplots(1,1, figsize=(18,10))
df=all_df_z0#.sample(n=100000)
#colormap, norm = plotto.gen_supermap(all_df_z0.fossilness)
a.scatter((ureg.gmass*df.mcri).to('Msun'),df.c200c, s=1, label='Magneticum Box0/mr, Box2b/hr, Box4/uhr',color='darkslategray') #c=rdf.redshift, s=2)
def sc(c0,a):
    return lambda x: c0*(x/1.e14)**a

for interp_observation_name in interp_observations:
    obs = interp_observations[interp_observation_name]
    x = np.logspace(obs.xrange[0],obs.xrange[1],3)
    if 'h' in obs.__dict__ and obs.h==True:
        x=x/.704
    y = sc(obs.c1e14,obs.B)(x)
    a.plot(x,y, linewidth=3, label=interp_observation_name, **{"linestyle":":"} if 'sim' in obs.__dict__ and  obs.sim else {})

x = np.logspace(11,15.2,4)
y = sc(4.5,-0.1)(x)

a.plot(x,y, linewidth=3,linestyle=":",color="black",label="This Work")


    

a.errorbar(pratt2005.m, pratt2005.c, yerr=pratt2005.cpm, fmt='+',color="red",label="Pratt et al. 2005")


a.errorbar(martinsson.m200.value,
               martinsson.c200c.value,
               xerr=[martinsson.m200.minus,martinsson.m200.plus ], 
               yerr=[martinsson.c200c.minus, martinsson.c200c.plus ],
               color='cyan',
               fmt='+', label="Martinsson et al. 2014")

for idx, row in meneghetti.iterrows():
    a.errorbar(row.M200c.value.to('Msun'),
               row.c200c.value,
               xerr=[[row.M200c.minus.to('Msun')], [row.M200c.plus.to('Msun')]], 
               yerr=[[row.c200c.plus], [row.c200c.minus]], 
               color='red',
               fmt='o', **{"label":'Meneghetti et al. 2014' } if idx==0 else {} )

for idx, row in clash.iterrows():
    a.errorbar(row.m200c.value.to('Msun'),
               row.c200c.value,
               xerr=[[row.m200c.minus.to('Msun')], [row.m200c.plus.to('Msun')]], 
               yerr=[[row.c200c.plus], [row.c200c.minus]], 
               color='magenta',
               fmt='+', **{"label":'Merten et al. 2015' } if idx==0 else {} )
    
for idx, row in mantz.iterrows():
    a.errorbar(row.m200.value.to('Msun'),
               row.c200.value,
               xerr=[[row.m200.minus.to('Msun')], [row.m200.plus.to('Msun')]], 
               yerr=[[row.c200.plus], [row.c200.minus]],
               color='green',
               fmt='+', **{"label":'Mantz et al. 2016' } if idx==0 else {} )

    
for idx, row in andrea_old.iterrows():
    a.errorbar(row.m200c.value,
               row.c200c.value,
               xerr=[[row.m200c.minus], [row.m200c.plus]], 
               yerr=[[row.c200c.plus], [row.c200c.minus]],
               color='blue',
               fmt='+', **{"label":'Biviano et al. 2017' } if idx==0 else {} )


    
set_Mc_lims(a)
fig.savefig('/home/moon/ragagnin/MagneticumMC/res/misuri.png',bbox_inches='tight')


In [ ]:


fig,a = plt.subplots(1,1, figsize=(18,10))
all_df_z0.fossilness = np.array(all_df_z0.fossilness, dtype=np.float32)
#display(all_df_z0.fossilness[~(all_df_z0.fossilness>=0)])
plotdf = all_df_z0[(all_df_z0.fossilness>0)&(all_df_z0.fossilness<np.inf)]
#all_df_z0=all_df_z0[all_df_z0.fossilness>0]#.sample(n=100)
#df = 
colormap, norm = plotto.gen_supermap(plotdf.fossilness,poutside=0.05)


c500 = plotdf.r5cc/plotdf.rcri*plotdf.c200c
m500 = ureg.gmass*plotdf.mcri*(500./200.)*(c500/plotdf.c200c)**3

#m500,c500 = convert_mass_and_concentration(500., 200.,df.c200c, ureg.gmass*df.mcri)
#display(m500.to('Msun'))
#display(c500)
#display(plotdf.fossilness)
scatter = a.scatter(m500.to('Msun'),c500, c= plotdf.fossilness, s=1, norm=norm, cmap=colormap) #c=rdf.redshift, s=2)
cbar_ax = fig.add_axes([0.1, 0.0, 0.82, 0.02])
colorbar = fig.colorbar(scatter, label="fossilness",cax=cbar_ax,  orientation='horizontal')

for idx, row in pratt.iterrows():
    color = colorbar.to_rgba(10.**(row.Deltam.value/2.5) )
    a.errorbar(row.M500.value, row.c500.value, 
               xerr=[[row.M500.minus], [row.M500.plus]],
               yerr=[[row.c500.minus], [row.c500.plus]],
                linewidth=5, color='black', ecolor='black',
               fmt='x')
"""    
for idx, row in pratt.iterrows():
    color = colorbar.to_rgba(10.**(row.Deltam.value/2.5) )
    a.errorbar(row.M500.value, row.c500.value, 
               xerr=[[row.M500.minus], [row.M500.plus]],
               yerr=[[row.c500.minus], [row.c500.plus]],
               #c=color, ecolor=color, linewidth=3, 
               c=color, ecolor=color, linewidth=3, 
               fmt='x',**{"label":'Pratt et al. 2016' } if idx==0 else {} )


"""
andrea_fossil_mask = andrea.fossilness.value>4.

idrow = -1
for idx, row in andrea[andrea_fossil_mask].iterrows():
    idrow +=1
    a.scatter(row.M500c.value,
           row.c500c.value,
           #xerr=[[row.m200c.minus], [row.m200c.plus]], 
           #yerr=[[row.c200c.plus], [row.c200c.minus]], 
           c='black',
           linewidth=5, cmap=colormap, norm=norm,
           #fmt='o',
           **{"label":'Biviano et al. 2018, fossil>4' }if idrow==0 else {}  )
idrow = -1
for idx, row in andrea[~andrea_fossil_mask].iterrows():
    idrow +=1
    a.scatter(row.M500c.value,
           row.c500c.value,
           #xerr=[[row.m200c.minus], [row.m200c.plus]], 
           #yerr=[[row.c200c.plus], [row.c200c.minus]], 
           c='gray',
           linewidth=5, cmap=colormap, norm=norm,
           #fmt='o',
           **{"label":'Biviano et al. 2018, non fossil<4' }if idrow==0 else {}  )

fmts ="soxo"
colors=["maroon","fuchsia","pink","maroon"]
"# ^<>v12348sp"
              
for idx, row in fossil.iterrows():
    #if idx==3: break
    a.errorbar(row.M500.value,
               row.c500.value,
               xerr=[[row.M500.minus], [row.M500.plus]], 
               yerr=[[row.c500.plus], [row.c500.minus]], 
               color='black',# linewidth=5, # colors[idx],
               fmt=fmts[idx],  label='%s (%s)'%(row.author.value, row.cluster.value) )

pratt2005.c500 = convert_concentration(500., 200.,pratt2005.c)
pratt2005.c500pm = pratt2005.c500 - convert_concentration(500., 200.,pratt2005.c - pratt2005.cpm)
pratt2005.m500 =  convert_mass(500., 200., pratt2005.c500, pratt2005.c,  pratt2005.m)
a.errorbar(pratt2005.m500, pratt2005.c500, yerr=pratt2005.c500pm, fmt='+',color="gray",label="Pratt et al. 2005")
for idx, row in bartolucci.iterrows():
    #if idx==3: break
    a.errorbar(row.m500.value,
               row.c500.value,
               xerr=[[row.m500.minus], [row.m500.plus]], 
               yerr=[[row.c500.plus], [row.c500.minus]], 
               color='gray',# linewidth=5, # colors[idx],
               fmt='x', **{ "label":'Bartolucci et al. 2018'} if idx==0 else {}  )

set_Mc_lims(a)
a.set_xlabel('M500c [Msun]')
a.set_ylabel('c500c')
fig.savefig('/home/moon/ragagnin/MagneticumMC/res/lib_Mc500cPratt.png',bbox_inches='tight')


In [ ]:
cm500,cc500 = convert_mass_and_concentration(500., 200.,df.c200c, ureg.gmass*df.mcri)


In [ ]:
all_df = simul ("""
    select  fof.mcri, fof.tgas0, pp.* ,snap.a, fof.r5cc, fof.rvir, fof.m5cc, fof.glen, fof.rcri, fof.id_cluster as fof_id_cluster, pp.id_cluster as pp_id_cluster
    from pp
    inner join fof on fof.snap_id=pp.snap_id and fof.id_cluster = pp.id_cluster
    inner join snap on  pp.snap_id = snap.id
    inner join simulation on simulation.id = snap.simulation_id
    and instr( simulation.name,'_dm')<=0
     
   -- and snap.tag='z0'
    --and snap.id= 5
    """)
qz="""
    select  fof.mcri, fof.tgas0, pp.* , snap.a,fof.r5cc, fof.rvir, fof.m5cc, fof.glen, fof.rcri, fof.id_cluster as fof_id_cluster, pp.id_cluster as pp_id_cluster
    from pp
    inner join fof on fof.snap_id=pp.snap_id and fof.id_cluster = pp.id_cluster
    inner join snap on  pp.snap_id = snap.id
    inner join simulation on simulation.id = snap.simulation_id
    and instr( simulation.name,'_dm')<=0
     
    and snap.tag='%s'
    --and snap.id= 5
    """

qzb="""
    select  fof.resolvness, fof.mcri, fof.tgas0, pp.* , snap.a,fof.r5cc, fof.rvir, fof.m5cc, fof.glen, fof.rcri, fof.id_cluster as fof_id_cluster, pp.id_cluster as pp_id_cluster
    from pp
    inner join fof on fof.snap_id=pp.snap_id and fof.id_cluster = pp.id_cluster
    inner join snap on  pp.snap_id = snap.id
    inner join simulation on simulation.id = snap.simulation_id
    and instr( simulation.name,'_dm')<=0
      and instr( simulation.name,'%s')>0
    and snap.tag='%s'
    --and snap.id= 5
    """
all_df_z0 = simul (qz%'z0')
all_df_z05 = simul (qz%'z05')
all_df_z1 = simul (qz%'z1')
all_df_z15 = simul (qz%'z15')
all_df_z2 = simul (qz%'z2')


box0_df_z0 = simul (qzb%('Box0','z0'))
box0_df_z05 = simul(qzb%('Box0','z05'))
box0_df_z1 = simul (qzb%('Box0','z1'))
box0_df_z15 = simul (qzb%('Box0','z15'))
box0_df_z2 = simul (qzb%('Box0','z2'))

box2_df_z0 = simul  (qzb%('Box2','z0'))
box2_df_z05 = simul  (qzb%('Box2','z05'))
box2_df_z1 = simul  (qzb%('Box2','z1'))
box2_df_z15 = simul  (qzb%('Box2','z15'))
box2_df_z2 = simul  (qzb%('Box2','z2'))

box4_df_z0 = simul (qzb%('Box4','z0'))
box4_df_z05 = simul (qzb%('Box4','z05'))
box4_df_z1 = simul (qzb%('Box4','z1'))
box4_df_z15 = simul(qzb%('Box4','z15'))
box4_df_z2 = simul (qzb%('Box4','z2'))

In [ ]:
import scipy.optimize

def fit_logc_logM(x,q,m):
    return m*(x-14.)+q
def topm(v,e):
    
    digite = int(np.abs(np.log10(e))+1)
    
    formattamelo = "%%.%df"%(digite)
    #print(v,e,formattamelo)
    stringola = formattamelo+" \pm "+formattamelo
    fstringola = stringola%(v,e)
    return fstringola

def linfit(xdata,ydata, bounds=[[0.1,10.],[0.1,10.]], f=fit_logc_logM):
    maskera =  (~np.isnan(ydata))
    if len(xdata.shape)==1:
        maskera&=~np.isnan(xdata)
        
    if len(xdata.shape)==2:
        
        for xdato in xdata:
            
            maskera&=~np.isnan(xdato)
        
    #display(xdata[maskera], ydata[maskera])
    xm=xdata.T[maskera]
    ym=ydata[maskera]
    # print(xm)
    #print(ym)
    v,pcov = scipy.optimize.curve_fit(f, xm.T, ym, bounds=bounds)
    perr = np.sqrt(np.diag(pcov))
    #print(v,10**v[0])
    
    s = topm(10.**v[0],10.**v[0]-10.**(v[0]-perr[0]))
    print(s)
    s = topm( v[1], perr[1])
    print(s)
    if len(v)>=3:
        print( topm( v[2], perr[2]))
    if len(v)>=4: print( topm( v[3], perr[3]))
    if len(v)>=5: print( topm( v[4], perr[4]))
    if len(v)>=6: print( topm( v[5], perr[5]))
    if len(v)>=7: print( topm( v[6], perr[6]))
    if len(v)>=8: print( topm( v[7], perr[7]))
    print()
    return v,perr
import numpy as np
def logc_logM(all_df):
    all_df_plot = all_df.sample(n=1000)
    logM = np.log10(all_df.mcri*1e10/.704)
    logc = np.log10(all_df.c200c)
    logMplot = np.log10(all_df_plot.mcri*1e10/.704)
    logcplot = np.log10(all_df_plot.c200c)
    bins=50
    logM_n = logc_n = np.histogram(logM, bins=bins )[0]
    logM_t = np.histogram(logM, weights=logM, bins=bins )[0]
    logM_b = logM_t/logM_n
    #logM_b = 0.5*(hbins[1:]+hbins[:-1])
    logc_t =  np.histogram(logM, weights=logc , bins=bins )[0]
    logc_avg = logc_t/logc_n
    v,perr = linfit(logM_b,logc_avg, bounds=np.array([[-1.,5.],[-1.,5.]]).T)
    #fig,a = plt.subplots(1,1, figsize=(18,10))
    #a.scatter(logMplot,logcplot)
    #a.plot(logM_b, logc_avg,color='red')
    #v,perr = linfit(logM_b,logc_avg, bounds=np.array([[-1.,5.],[-1.,5.]]).T)
    
    #a.plot(logM_b, fit_logc_logM(logM_b,v[0],v[1]),color='red')
    #display(logM_b, logc_avg)

logc_logM(all_df_z0)

logc_logM(all_df_z05)

logc_logM(all_df_z1)

logc_logM(all_df_z15)

logc_logM(all_df_z2)

logc_logM(all_df)   

In [ ]:
np.median(all_df.fossilness[ all_df.fossilness>0])

In [ ]:
import scipy.optimize
import numpy as np

def fit_logc_logM_loga(x,logA,B,C):
    logmassa=x[0]
    loga=x[1]
    return B*(logmassa-14.)+logA+(loga+np.log10(1.34))*C

def fit_logc_logM_loga_logfos(x,logA,B,C,D):
    logmassa=x[0]
    loga=x[1]
    logfos=x[2]
    return B*(logmassa-14.)+logA+(loga+np.log10(1.34))*C+D*(logfos-0.73)

def fit_logc_logM_loga_logfos_2sl(x,logA,B,C,D,F0,D2):
    logmassa=x[0]
    loga=x[1]
    logfos=x[2]
    return B*(logmassa-14.)+logA+(loga+np.log10(1.34))*C+D*(logfos-0.73)+(D2-D)*(np.log10(1.+(10.**logfos)/F0))


 
def fit_logc_logM_logabullo(x,logA,B):
    logmassa=x[0]
    loga=x[1]
    return B*(logmassa-14.)+logA+(loga+np.log10(1.34))

 
def logc_logM_logas(all_dfs,f = fit_logc_logM_loga, bounds=None):
    fittamix=[]
    fittamiy=[]
    bins=50
    for all_df in all_dfs:
        loga = np.log10( all_df.a[0])
        logM = np.log10(all_df.mcri*1e10/.704)
        logc = np.log10(all_df.c200c)
        if len(logc)<bins:
            continue
        logM_n = logc_n = np.histogram(logM, bins=bins )[0]
        logM_t = np.histogram(logM, weights=logM, bins=bins )[0]
        logM_b = logM_t/logM_n
    
        logc_t =  np.histogram(logM, weights=logc , bins=bins )[0]
        logc_avg = logc_t/logc_n 
        #display(logM.shape, bins)
        for bin in range(bins):
            
            fittamix.append([logM_b[bin],loga])
            fittamiy.append(logc_avg[bin])
    v,perr = linfit(np.array(fittamix).T,np.array(fittamiy), bounds=bounds,f=f)
    return
    for all_df in all_dfs:
        
        
        loga = np.log10( all_df.a[0])
        logM = np.log10(all_df.mcri*1e10/.704)
        logc = np.log10(all_df.c200c)
     
        logM_n = logc_n = np.histogram(logM, bins=bins )[0]
        logM_t = np.histogram(logM, weights=logM, bins=bins )[0]
        logM_b = logM_t/logM_n
        
        
        fig,a = plt.subplots(1,1, figsize=(18,10))
        if(len(all_df)>1000):
            all_df_plot = all_df.sample(n=1000)
        else:
            all_df_plot = all_df
        logMplot = np.log10(all_df_plot.mcri*1e10/.704)
        logcplot = np.log10(all_df_plot.c200c)
        
        a.scatter(logMplot,logcplot)
        asi = np.zeros(bins)+loga
        data_to_fit = np.array([logM_b,asi])
        ydata = fit_logc_logM_loga(data_to_fit,v[0],v[1],v[2])
        a.plot(logM_b,ydata,color='red')
    #a.plot(logM_b, logc_avg,color='red')
    #v,perr = linfit(logM_b,logc_avg, bounds=np.array([[-1.,5.],[-1.,5.]]).T)
    
    #a.plot(logM_b, fit_logc_logM(logM_b,v[0],v[1]),color='red')
    #display(logM_b, logc_avg)

    
#print('c = A (M/1e14Msun)^B ')
#logc_logM_loga([all_df])

print('c = A (M/1e14Msun)^B (1.34/1+z)^C ')
logc_logM_logas([box0_df_z0, box0_df_z05, box0_df_z1,box0_df_z15 ,box0_df_z2,
                box2_df_z0, box2_df_z05, box2_df_z1,box2_df_z15 ,box2_df_z2,
                box4_df_z0, box4_df_z05, box4_df_z1,box4_df_z15 ,box4_df_z2]
                ,bounds= np.array([[-5.,5.],[-5.,5.],[-5.,1.]]).T)



print('c = A (M/1e14Msun)^B a ')
logc_logM_logas([box0_df_z0, box0_df_z05, box0_df_z1,box0_df_z15 ,box0_df_z2,
                box2_df_z0, box2_df_z05, box2_df_z1,box2_df_z15 ,box2_df_z2,
                box4_df_z0, box4_df_z05, box4_df_z1,box4_df_z15 ,box4_df_z2],f=fit_logc_logM_logabullo,bounds = np.array([[-5.,5.],[-5.,5.] ]).T)



 
def logc_logM_logas_logfos(all_dfs,f = fit_logc_logM_loga_logfos, bounds=None, maxfoss=None, minfoss=None):
    fittamix=[]
    fittamiy=[]
    xbins=20
    ybins=20
    for all_df in all_dfs:
        loga = np.log10( all_df.a[0])
        logM =  np.log10(all_df.mcri*1e10/.704)
        logc = np.log10(all_df.c200c)
        logfoss =  np.log10(all_df.fossilness)#np.log10(all_df.fossilness)
        isfinitem = np.isfinite(logfoss) &  np.isfinite(logM)
        if maxfoss is not None:
            isfinitem &=logfoss<np.log10(maxfoss)
        if minfoss is not None:
            isfinitem &=logfoss>np.log10(minfoss)
            
        logM = logM [isfinitem]
        logc = logc[isfinitem]
        logfoss = logfoss[isfinitem]
        if len(logc)<xbins:
            continue
        logfoss_n = logM_n = logc_n = np.histogram2d(logM, logfoss, bins=(xbins, ybins) )[0]
        logM_t = np.histogram2d(logM, logfoss,weights=logM, bins=(xbins, ybins) )[0] 
        logM_b = logM_t/logM_n
    
        logc_t =  np.histogram2d(logM, logfoss,weights=logc, bins=(xbins, ybins) )[0] 
        logc_avg = logc_t/logc_n  
        
        logfoss_t =   np.histogram2d(logM, logfoss,weights=logfoss, bins=(xbins, ybins) )[0] 
        logfoss_avg = logfoss_t/logfoss_n 
        #display(logM.shape, bins)
        for xbin in range(xbins):
            for ybin in range(ybins):
                if np.isfinite(logM_b[xbin][ybin]) and np.isfinite(logfoss_avg[xbin][ybin]) and np.isfinite(logc_avg[xbin][ybin]):
                    fittamix.append([logM_b[xbin][ybin],loga,logfoss_avg[xbin][ybin]])
                    fittamiy.append(logc_avg[xbin][ybin])
                #print (fittamix)
                #print(np.array(fittamix))
    v,perr = linfit(np.array(fittamix).T,np.array(fittamiy), bounds=bounds,f=f)
    return
    for all_df in all_dfs:
        
        
        loga = np.log10( all_df.a[0])
        logM = np.log10(all_df.mcri*1e10/.704)
        logc = np.log10(all_df.c200c)
     
        logM_n = logc_n = np.histogram(logM, bins=bins )[0]
        logM_t = np.histogram(logM, weights=logM, bins=bins )[0]
        logM_b = logM_t/logM_n
        
        
        fig,a = plt.subplots(1,1, figsize=(18,10))
        if(len(all_df)>1000):
            all_df_plot = all_df.sample(n=1000)
        else:
            all_df_plot = all_df
        logMplot = np.log10(all_df_plot.mcri*1e10/.704)
        logcplot = np.log10(all_df_plot.c200c)
        
        a.scatter(logMplot,logcplot)
        asi = np.zeros(bins)+loga
        data_to_fit = np.array([logM_b,asi])
        ydata = fit_logc_logM_loga(data_to_fit,v[0],v[1],v[2])
        a.plot(logM_b,ydata,color='red')
        

print('c = A (M/1e14Msun)^B (1.34/1+z)^C foss^D ')
      
logc_logM_logas_logfos([box0_df_z0, box0_df_z05, box0_df_z1,box0_df_z15 ,box0_df_z2,
                box2_df_z0, box2_df_z05, box2_df_z1,box2_df_z15 ,box2_df_z2,
                box4_df_z0, box4_df_z05, box4_df_z1,box4_df_z15 ,box4_df_z2],
                       bounds = np.array([[-5.,5.],[-5.,5.] ,[-5.,5.],[-5.,5.]]).T)

print('c = A (M/1e14Msun)^B (1.34/1+z)^C foss^D; foss<15 ')

logc_logM_logas_logfos([box0_df_z0, box0_df_z05, box0_df_z1,box0_df_z15 ,box0_df_z2,
                box2_df_z0, box2_df_z05, box2_df_z1,box2_df_z15 ,box2_df_z2,
                box4_df_z0, box4_df_z05, box4_df_z1,box4_df_z15 ,box4_df_z2],
                       bounds = np.array([[-5.,5.],[-5.,5.] ,[-5.,5.],[-5.,5.]]).T,maxfoss=20., minfoss=.5)


print('c = A (M/1e14Msun)^B (1.34/1+z)^C foss^D; foss>15 ')


logc_logM_logas_logfos([box0_df_z0, box0_df_z05, box0_df_z1,box0_df_z15 ,box0_df_z2,
                box2_df_z0, box2_df_z05, box2_df_z1,box2_df_z15 ,box2_df_z2,
                box4_df_z0, box4_df_z05, box4_df_z1,box4_df_z15 ,box4_df_z2],
                       bounds = np.array([[-5.,5.],[-5.,5.] ,[-5.,5.],[-5.,5.]]).T,minfoss=30.)


print('c = A (M/1e14Msun)^B (1.34/1+z)^C foss^D twosl; ')


logc_logM_logas_logfos([box0_df_z0, box0_df_z05, box0_df_z1,box0_df_z15 ,box0_df_z2,
                box2_df_z0, box2_df_z05, box2_df_z1,box2_df_z15 ,box2_df_z2,
                box4_df_z0, box4_df_z05, box4_df_z1,box4_df_z15 ,box4_df_z2],
                       bounds = np.array([[-5.,5.],[-5.,5.] ,[-5.,5.],[-5.,5.],[.01,200.],[-2.,2.]]).T,
                      f=fit_logc_logM_loga_logfos_2sl)



In [ ]:
from scipy import stats
c200c_bx = lambda df: 10.**fit_logc_logM_loga_logfos(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a), np.log10(df.fossilness) ],
                       np.log10( 4.98),-0.088 ,0.14 ,0.107              )
c200c_sx = lambda df: 10.**fit_logc_logM_loga_logfos(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a), np.log10(df.fossilness) ],
                       np.log10( 4.56),-0.097 ,0.116 ,0.239              )
c200c_dx = lambda df: 10.**fit_logc_logM_loga_logfos(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a), np.log10(df.fossilness) ],
                       np.log10( 5.95),-0.088 ,0.156 ,-0.004              )
c200c_2x = lambda df: 10.**fit_logc_logM_loga_logfos_2sl(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a), np.log10(df.fossilness) ],
                       np.log10( 8.6),-0.095 ,0.135 ,0.54, 3.0, -0.012              )

if True:
    all_df_sample = all_df#.sample(n=10000)    
    figy,ay = plt.subplots(1, 1, figsize=(14,10))
    #figy.suptitle(title)
    #ay.scatter(all_df_sample)
    ay.set_ylabel('c200')
    ay.set_xlabel('fossilness')
    ay.set_xscale('log')
    ay.set_xlim([1.,1e4])
    ay.set_yscale('log')
    ay.scatter(all_df_sample.fossilness, all_df_sample.c200c,s=1)#, c=all_df_sample.mcri)
    #display(all_df[(all_df_sample.fossilness>2 )&(all_df_sample.fossilness<3)])
    plotfossbins = np.logspace(-2.,4.,100)
    binsc200c_n = np.histogram(all_df.fossilness, bins=plotfossbins)[0]
    binsc200c_sx_t = np.histogram(all_df.fossilness, weights = c200c_sx(all_df), bins=plotfossbins)[0]
    binsc200c_dx_t = np.histogram(all_df.fossilness, weights = c200c_dx(all_df), bins=plotfossbins)[0]
    binsc200c_bx_t = np.histogram(all_df.fossilness, weights = c200c_bx(all_df), bins=plotfossbins)[0]
    binsc200c_2x_t = np.histogram(all_df.fossilness, weights = c200c_2x(all_df), bins=plotfossbins)[0]
    medians = stats.binned_statistic(all_df.fossilness, all_df.c200c, 'median', bins=plotfossbins)[0]
                                                                                                
    #display(plotfossbins, binsc200c_t, binsc200c_n)
    ay.plot(plotfossbins[1:]            ,medians, color='navy', label='median',linewidth=5 )
    ay.plot(plotfossbins[1:]            ,binsc200c_2x_t/binsc200c_n, color='pink',label=' two sloppes',linestyle=':' ,linewidth=5)
    ay.plot(plotfossbins[1:][plotfossbins[1:]<20.]            ,(binsc200c_sx_t/binsc200c_n)[plotfossbins[1:]<20.], color='red' , label='single slope, foss<20',linestyle=':',linewidth=5)
    #ay.plot(plotfossbins[1:][plotfossbins[1:]>20.]            ,(binsc200c_dx_t/binsc200c_n)[plotfossbins[1:]>20], color='blue', label='single slope, foss>15',linestyle=':',linewidth=5 )
    #ay.plot(plotfossbins[1:]            ,binsc200c_bx_t/binsc200c_n, color='magenta', label='single slope' ,linestyle=':',linewidth=5)
    figy.legend(loc=2)
    figy.savefig('/home/moon/ragagnin/MagneticumMC/res/fossilness_fits.png')#,bbox_inches='tight')


In [ ]:

import numpy as np
def plot_residuals(ax,magc,fitc):
    residui = (magc-fitc)/fitc
    
    ax.hist(residui[~np.isnan(residui)], 13, density=True,range=[-1.,1.],  facecolor='green', alpha=0.75)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    ax.text(0.2,.8,'N=%d'%(len(residui[~np.isnan(residui)])), 
            horizontalalignment='center',
      verticalalignment='center',
      transform=ax.transAxes)
    
def plot_all_residuals(df,magc, f, title='', mainplot=True):
    if mainplot:
        fig,a = plt.subplots(1,1,sharex=True)
        fig.suptitle(title)
        plot_residuals(a,magc, f(df))
    Nm=3
    fig,axes = plt.subplots(Nm,3,sharex=True, sharey=True,figsize=(6,5))
    fig.tight_layout()
    fig.suptitle(title)
    #massspace  = np.logspace(np.log10(np.min(df.mcri)), np.log10(np.max(df.mcri)),Nm+1)
    massspace  = [0., 1e3, 1e4, 1e5,1e6]
    zspace  = [0.,0.4,1.1,2.5]
    imass = -1
    iz=-1
    for a1 in axes:
        imass = imass+1
        #print("M", massspace[imass],massspace[imass+1], len(df[mask]))
        iz=-1
        for a in a1:
            
            mask = (df.mcri > massspace[imass]) &  (df.mcri < massspace[imass+1])
            iz=iz+1
            a.set_xticklabels([])
            a.set_yticklabels([])
            if (iz==0):
                a.set_xlabel('z<%.1f'%(zspace[iz+1]))
                if (imass==0):
                    a.set_ylabel('<%.0e Msun'%(massspace[imass+1]*1e10/.704),fontsize=8)
                
                elif imass<len(axes)-1:
                    a.set_ylabel('%.0e - %.0e Msun'%(massspace[imass]*1e10/.704,massspace[imass+1]*1e10/.704),fontsize=8)
                else:
                    a.set_ylabel('>%.0e Msun'%(massspace[imass]*1e10/.704),fontsize=8)
                
            elif iz<len(a1)-1:
                a.set_xlabel('%.1f<z<%.1f'%(zspace[iz],zspace[iz+1]))
            else:
                a.set_xlabel('z>%.1f'%(zspace[iz]))
        
            mask &= ((1./df.a-1.) > zspace[iz]) &  ((1./df.a-1.) < zspace[iz+1])
            
            subdf = df[mask]
            #print("Z",  zspace[iz],zspace[iz+1], len(subdf),(1./df[ (df.mcri > massspace[imass]) &  (df.mcri < massspace[imass+1])].a-1.).unique(),(1./subdf.a-1.).unique())
            plot_residuals(a,subdf.c200c, f(subdf))
    fig.subplots_adjust(wspace=0, hspace=0,top=0.85)

    
 
   
plot_all_residuals(all_df,  df.c200c,
                   lambda df: 10.**fit_logc_logM(
                        np.log10(df.mcri*1e10/.704),
                        
                       np.log10(4.55) ,
                        -0.104,
                       
                   ), title='c200 = A M^B \n')  


plot_all_residuals(all_df,  df.c200c,
                   lambda df: 10.**fit_logc_logM_loga(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a)],
                        
                       np.log10(4.70) ,
                        -0.117,
                       0.17
                   ), title='c200 = A M^B (1.34/1+z)^B\n'
                  
                  )



def plot_all_residuals_foss_bins(all_df, magc, c200c, title='', logfossbins = None):
    if logfossbins is None:
        logfossbins = [[0., 5.],[5., 100.],[100. ,1000]]
    ibin=-1
    
    plot_all_residuals(all_df, magc, c200c, title=title+'all foss')
    for logfossbin in logfossbins:
        ibin+=1
        if ibin==0:
            apptitle = ' foss<%.0f'%(logfossbin[1])
        elif ibin<len(logfossbins)-1:
            apptitle = ' %.0f<foss<%.0f'%(logfossbin[0],logfossbin[1])
        else:
            apptitle = ' foss>%.0f'%(logfossbin[0])
        df_mask = (all_df.fossilness>logfossbin[0])&(all_df.fossilness<logfossbin[1])
        plot_all_residuals(all_df[df_mask], magc[df_mask], c200c, title=title+apptitle, mainplot=False)


plot_all_residuals_foss_bins(all_df,  all_df.c200c,
                   lambda df: 10.**fit_logc_logM_loga_logfos(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a), np.log10(df.fossilness) ],
                       np.log10( 4.56),
-0.097 ,
0.117 ,
0.239              ), title='c200 = A M^B (1.34/1+z)^B (foss/1.31)^D\n',
                  logfossbins =[ [0.,5.], [5., 20.] ]
                  )


plot_all_residuals_foss_bins(all_df,  df.c200c,
                   lambda df: 10.**fit_logc_logM_loga_logfos_2sl(
                       [ np.log10(df.mcri*1e10/.704), np.log10(df.a), np.log10(df.fossilness) ],
                      
np.log10(8.6) ,
-0.095,
0.135 ,
0.54 ,
3.0 ,
-0.012            ), title='c200 = A M^B (1.34/1+z)^B (foss/1.31)^D  (1+foss/foss0)^(D-E) \n'
                  
                  )




In [ ]:
len(all_df[(all_df.fossilness>2 )&(all_df.fossilness<3))

In [ ]:

xdf0 = simul ("""
select fof.id_cluster,1e10*fof.mcri,snap.id as snap_id, snap.tag,simulation.name from fof 
inner join galaxy on galaxy.id_cluster = fof.id_cluster and galaxy.snap_id = fof.snap_id
inner join snap on snap.id = fof.snap_id
inner join simulation on simulation.id = snap.simulation_id
 where  
 fof.mcri>.8e15/1e10 and
  galaxy.msub>fof.mcri*0.1

""")

xdf=xdf0


In [ ]:


xdf['ngalaxies'] = xdf.groupby(['id_cluster','snap_id']).count().reset_index()['name']
xdf[xdf['ngalaxies'] >= 3] 


In [ ]:
fig,a = plt.subplots(1,1, figsize=(18,10))
df=all_df_z0#.sample(n=100000)
colormap, norm = plotto.gen_supermap(all_df_z0.fossilness)
scatter = a.scatter((ureg.gmass*df.mcri).to('Msun'),df.c200c, s=1, label='Magneticum Box0/mr, Box2b/hr, Box4/uhr',c=df.fossilness,cmap=colormap, norm=norm) #c=rdf.redshift, s=2)
cbar_ax = fig.add_axes([0.1, 0.0, 0.82, 0.02])
colorbar = fig.colorbar(scatter, label="fossilness",cax=cbar_ax,  orientation='horizontal')
def sc(c0,a):
    return lambda x: c0*(x/1.e14)**a

for interp_observation_name in interp_observations:
    obs = interp_observations[interp_observation_name]
    x = np.logspace(obs.xrange[0],obs.xrange[1],3)
    if 'h' in obs.__dict__ and obs.h==True:
        x=x/.704
    y = sc(obs.c1e14,obs.B)(x)
    a.plot(x,y, linewidth=3, label=interp_observation_name, **{"linestyle":":"} if 'sim' in obs.__dict__ and  obs.sim else {})

x = np.logspace(11,15.2,4)
y = sc(4.5,-0.1)(x)

a.plot(x,y, linewidth=3,linestyle=":",color="black",label="This Work")


    

a.errorbar(pratt2005.m, pratt2005.c, yerr=pratt2005.cpm, fmt='+',color="red",label="Pratt et al. 2005")


a.errorbar(martinsson.m200.value,
               martinsson.c200c.value,
               xerr=[martinsson.m200.minus,martinsson.m200.plus ], 
               yerr=[martinsson.c200c.minus, martinsson.c200c.plus ],
               color='cyan',
               fmt='+', label="Martinsson et al. 2014")

for idx, row in meneghetti.iterrows():
    a.errorbar(row.M200c.value.to('Msun'),
               row.c200c.value,
               xerr=[[row.M200c.minus.to('Msun')], [row.M200c.plus.to('Msun')]], 
               yerr=[[row.c200c.plus], [row.c200c.minus]], 
               color='red',
               fmt='o', **{"label":'Meneghetti et al. 2014' } if idx==0 else {} )

for idx, row in clash.iterrows():
    a.errorbar(row.m200c.value.to('Msun'),
               row.c200c.value,
               xerr=[[row.m200c.minus.to('Msun')], [row.m200c.plus.to('Msun')]], 
               yerr=[[row.c200c.plus], [row.c200c.minus]], 
               color='magenta',
               fmt='+', **{"label":'Merten et al. 2015' } if idx==0 else {} )
    
for idx, row in mantz.iterrows():
    a.errorbar(row.m200.value.to('Msun'),
               row.c200.value,
               xerr=[[row.m200.minus.to('Msun')], [row.m200.plus.to('Msun')]], 
               yerr=[[row.c200.plus], [row.c200.minus]],
               color='green',
               fmt='+', **{"label":'Mantz et al. 2016' } if idx==0 else {} )

    
for idx, row in andrea_old.iterrows():
    a.errorbar(row.m200c.value,
               row.c200c.value,
               xerr=[[row.m200c.minus], [row.m200c.plus]], 
               yerr=[[row.c200c.plus], [row.c200c.minus]],
               color='blue',
               fmt='+', **{"label":'Biviano et al. 2017' } if idx==0 else {} )


for idx, row in pratt.iterrows():
    #color = colorbar.to_rgba(10.**(row.Deltam.value/2.5) )
    a.errorbar(row.M200.value, row.c200.value, 
               xerr=[[row.M200.minus], [row.M200.plus]],
               yerr=[[row.c200.minus], [row.c200.plus]],
                linewidth=3, color='black', ecolor='black',
               fmt='x')
fmts ="soxo"
for idx, row in fossil.iterrows():
    #if idx==3: break
    a.errorbar(row.M200.value,
               row.c200.value,
               xerr=[[row.M200.minus], [row.M200.plus]], 
               yerr=[[row.c200.plus], [row.c200.minus]], 
               color='black', linewidth=3, # colors[idx],
               fmt=fmts[idx],  label='%s (%s)'%(row.author.value, row.cluster.value) )
    #display(row)
#pratt2005.c500 = convert_concentration(500., 200.,pratt2005.c)
#pratt2005.c500pm = pratt2005.c500 - convert_concentration(500., 200.,pratt2005.c - pratt2005.cpm)
#pratt2005.m500 =  convert_mass(500., 200., pratt2005.c500, pratt2005.c,  pratt2005.m)
#a.errorbar(pratt2005.m500, pratt2005.c500, yerr=pratt2005.c500pm, fmt='+',color="gray",label="Pratt et al. 2005")


    
set_Mc_lims(a)
a.set_ylim([1.,30.])
fig.savefig('/home/moon/ragagnin/MagneticumMC/res/misuri_200.png',bbox_inches='tight')


In [ ]:
fig,a = plt.subplots(1,1, figsize=(5,5))
a.hist(all_df.fossilness, np.logspace(-2,4,40), normed=True, alpha=0.75,histtype='step')
a.hist(all_df.fossilness[(all_df.mcri>1e1)], np.logspace(-2,4,40), normed=1,  alpha=0.75,histtype='step')
a.hist(all_df.fossilness[(all_df.mcri<1e1)], np.logspace(-2,4,40), normed=1,  alpha=0.75,histtype='step')
a.set_xscale('log')


In [ ]:
box4_df_z0 = simul (qzb%('Box4','z0'))
box4_df_z05 = simul (qzb%('Box4','z05'))
box4_df_z1 = simul (qzb%('Box4','z1'))
box4_df_z15 = simul(qzb%('Box4','z15'))
box4_df_z2 = simul (qzb%('Box4','z2'))

In [ ]:
display(
    len(box4_df_z0[box4_df_z0.glen>1e4]),
    len(box4_df_z05[box4_df_z05.glen>1e4]),
    len(box4_df_z1[box4_df_z1.glen>1e4]),
    len(box4_df_z15[box4_df_z15.glen>1e4]),
    len(box4_df_z2[box4_df_z2.glen>1e4]))

#.sort_values('glen',ascending=True).glen

In [ ]:
Box2_df_z0 = simul ("""
    select  fof.* from fof
    inner join snap on  fof.snap_id = snap.id
    inner join simulation on simulation.id = snap.simulation_id
    and instr( simulation.name,'_dm')<=0
      and instr( simulation.name,'%s')>0
    and snap.tag='%s'
    --and snap.id= 5
    """%('Box2','z0'))


In [200]:
Box4_df= simul ("""
    select  fof.*, pp.* from fof
    inner join pp on pp.snap_id = fof.snap_id and pp.id_cluster = fof.id_cluster
    inner join snap on  fof.snap_id = snap.id
    inner join simulation on simulation.id = snap.simulation_id
    and instr( simulation.name,'_dm')<=0
      and instr( simulation.name,'%s')>0
     and snap.tag='z2'
    --and snap.id= 5
    """%('Box4'))

In [196]:
Box4_df.sort_values('glen').glen

1844    7307   
1843    7389   
1842    7397   
1841    7403   
1840    7514   
1839    7587   
1838    7664   
1837    7694   
1836    7708   
1835    7741   
1834    7793   
1833    7830   
1832    7842   
1831    7873   
1830    7882   
1829    7882   
1828    7901   
1827    7950   
1826    7953   
1825    8084   
1824    8112   
1823    8126   
1822    8146   
1821    8153   
1820    8157   
1819    8235   
1818    8242   
1817    8255   
1816    8308   
1815    8309   
        ...    
29      727667 
28      735025 
27      743374 
26      770881 
25      834242 
24      854397 
23      868988 
22      905369 
21      982804 
20      995303 
19      1063087
18      1187360
17      1286123
16      1315215
15      1358876
14      1576189
13      1748215
12      1861741
11      1927164
10      2025218
9       2141029
8       2247973
7       2407655
6       3447835
5       4111449
4       4215216
3       4321458
2       5814285
1       6700767
0       9117815
Name: glen, Length: 1845